### Instalar librerias

In [ ]:
!pip install git+https://github.com/openai/whisper.git
!pip install torch
!pip install pydub
!apt-get install ffmpeg
!pip install --upgrade openai

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-1g_z7q13
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-1g_z7q13
  Resolved https://github.com/openai/whisper.git to commit 279133e3107392276dc509148da1f41bfb532c7e
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.4/209.4 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 33.1 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20231117-py3-none-any.whl size=802819 sha256=b7f8839fd7e46c35bc6ddac1fbdb5a128b8e50153d9a648e659d3f1b0a95f010
  Stored in directory: /tmp/pip-ephem-wheel-cache-b09zjjdo/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper
Reading package lists... Done
Building dependency tree... Done
Reading state i

## Importar librerias

In [ ]:
import os
from openai import OpenAI
import pandas as pd
import whisper
import os
from pydub import AudioSegment
import subprocess
import math
from datetime import datetime

In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Unificar audios


In [ ]:
# Ruta de la carpeta con los archivos de audio
audios_path = 'Tu_ruta_aca'

In [ ]:
# Convertir archivos M4A a MP4
m4a_files = [f for f in os.listdir(audios_path) if f.endswith('.m4a')]

for m4a_file in m4a_files:
    input_file = os.path.join(audios_path, m4a_file)
    mp4_file = os.path.splitext(m4a_file)[0] + '.mp4'
    output_file = os.path.join(audios_path, mp4_file)

    # Comando FFmpeg para convertir M4A a MP4
    subprocess.call(['ffmpeg', '-i', input_file, '-c', 'copy', output_file])

In [ ]:
# Lista de archivos mp4 en la carpeta
audio_files = sorted([f for f in os.listdir(audios_path) if f.endswith('.mp4')], key=lambda x: int(x.split('.')[0]))

# Crear un objeto de audio unificado
unified_audio = AudioSegment.empty()

In [ ]:
# Unir los archivos en uno solo
for audio_file in audio_files:
    audio_path = os.path.join(audios_path, audio_file)
    audio = AudioSegment.from_file(audio_path, format='mp4')
    unified_audio += audio

# Guardar el archivo unificado
unified_audio_path = os.path.join(audios_path, "audio_unificado.mp4")
unified_audio.export(unified_audio_path, format='mp4')

print(f"Archivo unificado guardado en: {unified_audio_path}")

Archivo unificado guardado en: /content/drive/MyDrive/Proyectos/Transcribir a texto/Entrada/audio_unificado.mp4


## Estandarizacion de audios

In [ ]:
# Ruta de la carpeta donde se guardarán los audios divididos
output_path = 'Tu_ruta'

# Duración en milisegundos (5 minutos = 300000 ms)
chunk_length_ms = 5 * 60 * 1000

# Cargar el archivo de audio unificado
unified_audio = AudioSegment.from_file(unified_audio_path, format='mp4')

# Calcular cuántos fragmentos de 5 minutos se necesitan
num_chunks = math.ceil(len(unified_audio) / chunk_length_ms)

In [ ]:
# Dividir y guardar los fragmentos
for i in range(num_chunks):
    start_time = i * chunk_length_ms
    end_time = min((i + 1) * chunk_length_ms, len(unified_audio))
    audio_chunk = unified_audio[start_time:end_time]

    chunk_filename = f"{i+1}.mp4"
    chunk_path = os.path.join(output_path, chunk_filename)

    # Guardar el fragmento
    audio_chunk.export(chunk_path, format='mp4')
    print(f"Guardado: {chunk_filename}")

Guardado: 1.mp4
Guardado: 2.mp4
Guardado: 3.mp4
Guardado: 4.mp4
Guardado: 5.mp4
Guardado: 6.mp4
Guardado: 7.mp4
Guardado: 8.mp4
Guardado: 9.mp4
Guardado: 10.mp4
Guardado: 11.mp4
Guardado: 12.mp4
Guardado: 13.mp4
Guardado: 14.mp4
Guardado: 15.mp4
Guardado: 16.mp4
Guardado: 17.mp4
Guardado: 18.mp4
Guardado: 19.mp4


In [ ]:
# Eliminar todos los archivos en la carpeta 'Entrada'
for file in os.listdir(audios_path):
    file_path = os.path.join(audios_path, file)
    try:
        if os.path.isfile(file_path):
            os.remove(file_path)
            print(f"Eliminado: {file_path}")
    except Exception as e:
        print(f"Error al eliminar {file_path}: {e}")

Eliminado: /content/drive/MyDrive/Proyectos/Transcribir a texto/Entrada/1.m4a
Eliminado: /content/drive/MyDrive/Proyectos/Transcribir a texto/Entrada/2.m4a
Eliminado: /content/drive/MyDrive/Proyectos/Transcribir a texto/Entrada/3.m4a
Eliminado: /content/drive/MyDrive/Proyectos/Transcribir a texto/Entrada/1.mp4
Eliminado: /content/drive/MyDrive/Proyectos/Transcribir a texto/Entrada/2.mp4
Eliminado: /content/drive/MyDrive/Proyectos/Transcribir a texto/Entrada/3.mp4
Eliminado: /content/drive/MyDrive/Proyectos/Transcribir a texto/Entrada/audio_unificado.mp4


## Transcribir audios

In [ ]:
# Configurar la API de OpenAI
OPENAI_API_KEY = 'Tu_clave_de_openia'
# Configurar el cliente asíncrono
client = OpenAI(
    api_key=OPENAI_API_KEY
)

In [ ]:
# Ruta de los audios estandarizados (donde están los archivos de audio de 5 minutos)
audios_path = 'Tu_ruta_aca'

# Ruta donde se guardarán los archivos de texto
text_output_path = 'Tu_ruta_aca'

# Cargar el modelo Whisper
model = whisper.load_model("base")

100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 87.6MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:146: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [ ]:
# Función para transcribir audios y mejorar la redacción
def transcribe_and_improve_audios(audios_path):
    n = 0
    unified_transcription = ""

    # Obtener lista de archivos de audio en la carpeta
    audio_files = sorted(
        [f for f in os.listdir(audios_path) if f.endswith('.mp4')],
        key=lambda x: int(os.path.splitext(x)[0])
    )

    for audio_file in audio_files:
        audio_path = os.path.join(audios_path, audio_file)

        # Verificar si el archivo de audio existe
        if not os.path.exists(audio_path):
            print(f"Archivo de audio no encontrado: {audio_path}")
            continue

        try:
            # Transcribir el audio usando tu modelo (por ejemplo, Whisper)
            result = model.transcribe(audio_path, fp16=False)
            text = result['text']

            print(f"Audio transcrito: {audio_path}")
            print(f"Texto: {text}")

            # Mejorar la redacción usando la API de OpenAI
            retry_count = 0
            max_retries = 2  # Número máximo de intentos

            while retry_count < max_retries:
                try:
                    improved_text = improve_transcription(text)
                    # Si la mejora es exitosa, salimos del bucle de reintento
                    break
                except Exception as e:
                    retry_count += 1
                    print(f"Error al mejorar la transcripción: {e}")
                    if retry_count < max_retries:
                        print(f"Reintentando mejorar la transcripción ({retry_count}/{max_retries})...")
                    else:
                        print("No se pudo mejorar el texto después de varios intentos.")
                        improved_text = "No se pudo mejorar el texto debido a un error."

            # Añadir el texto mejorado al archivo unificado
            unified_transcription += f"\n\n=== Transcripción de {audio_file} ===\n\n{improved_text}\n"

        except Exception as e:
            print(f"Error al transcribir el audio {audio_file}: {e}")

    # Guardar el archivo unificado con todas las transcripciones mejoradas
    today_date = datetime.today().strftime('%Y-%m-%d')
    unified_txt_path = os.path.join(text_output_path, f'transcripcion_unificada_mejorada_{today_date}.txt')

    with open(unified_txt_path, 'w', encoding='utf-8') as f:
        f.write(unified_transcription)
    print(n)
    print(f"Transcripción unificada mejorada guardada en: {unified_txt_path}")

    # Generar el resumen final
    summary = generate_summary(unified_transcription)

    # Guardar el resumen en un archivo
    summary_txt_path = os.path.join(text_output_path, f'resumen_{today_date}.txt')

    with open(summary_txt_path, 'w', encoding='utf-8') as f:
        f.write(summary)

    print(f"Resumen guardado en: {summary_txt_path}")

# Función para mejorar la transcripción utilizando la API de OpenAI
def improve_transcription(text):
    prompt = f"""He transcrito un fragmento de una reunión donde un líder de área explica un macroproceso. El texto puede contener errores de transcripción, interrupciones y preguntas de otros participantes.

Tu tarea es:

- Corregir los errores de transcripción y mejorar la redacción donde sea necesario.
- Ignorar las interrupciones y preguntas que no aporten al contenido principal.
- Unificar la información proporcionada por el líder del área sobre el macroproceso.

Fragmento de la transcripción:

{text}

Por favor, proporciona el texto corregido y mejorado a continuación."""

    # Llamada a la API de OpenAI según tu formato específico
    completion = client.chat.completions.create(
        model="gpt-4o-mini",  # Asegúrate de que este es el modelo correcto
        messages=[
            {
                "role": "user",
                "content": f"{prompt}",
            },
        ],
    )

    # Procesar la respuesta de la API
    improved_text = completion.choices[0].message.content.strip()
    return improved_text

# Función para generar el resumen
def generate_summary(unified_transcription):
    prompt = f"""He recopilado la siguiente transcripción mejorada de una reunión donde un líder de área explica un macroproceso:

{unified_transcription}

Por favor, genera un resumen claro y conciso de los puntos principales discutidos por el líder del área."""

    # Llamada a la API de OpenAI según tu formato específico
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "user",
                "content": f"{prompt}",
            },
        ],
    )

    # Procesar la respuesta de la API
    summary = completion.choices[0].message.content.strip()
    return summary

In [ ]:
# Llamar a la función para transcribir y mejorar los audios
transcribe_and_improve_audios(audios_path)

Audio transcrito: /content/drive/MyDrive/Proyectos/Transcribir a texto/Audios estandarizados/1.mp4
Texto:  Malianza sur ya con referencia de las aplicaciones, ya os hemos dicho la necesidad de aplicar, pero en la que está a la hora de estar, ya estamos implementando, no, no, nosotros estamos, ya tenemos información, también no es capaz de validar, es como, este son fotos atalitales, de vez porque si te has dicho que la distancia de campo, te sé cómo lo tienes, te has creado, como ellos van, recorre, mira, ni se aquí malesa, se juntan en día, jueves, diseñen, bueno, te muro necesidad para explicar a 100 semanas, y antes, en función de eso, ya viene a salir a necesidad, se ponen de acuerdo con las que es producción, los dos de la gente de este de ese, y a viene y le hace una necesidad para aplicaciones a él, entonces allá le da punta, eso lo hacía ahí, porque si hubo ahora va a ser a lo largo, o sea, lo van a hacer, punta la necesidad de nuestros compañeros, y ahora viene y se lo mandan 

In [ ]:
# Eliminar todos los archivos en la carpeta 'Entrada'
for file in os.listdir(audios_path):
    file_path = os.path.join(audios_path, file)
    try:
        if os.path.isfile(file_path):
            os.remove(file_path)
            print(f"Eliminado: {file_path}")
    except Exception as e:
        print(f"Error al eliminar {file_path}: {e}")

Eliminado: /content/drive/MyDrive/Proyectos/Transcribir a texto/Audios estandarizados/1.mp4
Eliminado: /content/drive/MyDrive/Proyectos/Transcribir a texto/Audios estandarizados/2.mp4
Eliminado: /content/drive/MyDrive/Proyectos/Transcribir a texto/Audios estandarizados/3.mp4
Eliminado: /content/drive/MyDrive/Proyectos/Transcribir a texto/Audios estandarizados/4.mp4
Eliminado: /content/drive/MyDrive/Proyectos/Transcribir a texto/Audios estandarizados/5.mp4
Eliminado: /content/drive/MyDrive/Proyectos/Transcribir a texto/Audios estandarizados/6.mp4
Eliminado: /content/drive/MyDrive/Proyectos/Transcribir a texto/Audios estandarizados/7.mp4
Eliminado: /content/drive/MyDrive/Proyectos/Transcribir a texto/Audios estandarizados/8.mp4
Eliminado: /content/drive/MyDrive/Proyectos/Transcribir a texto/Audios estandarizados/9.mp4
Eliminado: /content/drive/MyDrive/Proyectos/Transcribir a texto/Audios estandarizados/10.mp4
Eliminado: /content/drive/MyDrive/Proyectos/Transcribir a texto/Audios estandar